In [10]:
# %pip install https://github.com/amarquand/PCNtoolkit/archive/refs/tags/v1.alpha.zip
# %pip install graphviz

In [11]:
import os

import pandas as pd
from pcntoolkit.dataio.norm_data import NormData
from pcntoolkit.normative_model.norm_conf import NormConf
from pcntoolkit.normative_model.norm_hbr import NormHBR
from pcntoolkit.regression_model.hbr.hbr_conf import HBRConf
from pcntoolkit.regression_model.hbr.likelihood import NormalLikelihood
from pcntoolkit.regression_model.hbr.prior import make_prior
from pcntoolkit.util.runner import Runner
from pcntoolkit.regression_model.blr.blr_conf import BLRConf
from pcntoolkit.normative_model.norm_blr import NormBLR
from pcntoolkit.regression_model.hbr.likelihood import SHASHbLikelihood
resources_dir = "resources"
abs_path = os.path.abspath(resources_dir)
data_dir = os.path.join(abs_path, "data")
os.makedirs(data_dir, exist_ok=True)

In [12]:
# If you are running this notebook for the first time, you need to download the dataset from github.
# If you have already downloaded the dataset, you can comment out the following line

pd.read_csv(
    "https://raw.githubusercontent.com/predictive-clinical-neuroscience/PCNtoolkit-demo/refs/heads/main/data/fcon1000.csv"
).to_csv(os.path.join(data_dir, "fcon1000.csv"), index=False)
data = pd.read_csv(os.path.join(data_dir, "fcon1000.csv"))
covariates = ["age"]
batch_effects = ["sex", "site"]
response_vars = "lh_G_temp_sup-G_T_transv_thickness,lh_G_temp_sup-Lateral_thickness,lh_G_temp_sup-Plan_polar_thickness".split(',')
# response_vars = "lh_G&S_frontomargin_thickness,lh_G&S_occipital_inf_thickness,lh_G&S_paracentral_thickness,lh_G&S_subcentral_thickness,lh_G&S_transv_frontopol_thickness,lh_G&S_cingul-Ant_thickness,lh_G&S_cingul-Mid-Ant_thickness,lh_G&S_cingul-Mid-Post_thickness,lh_G_cingul-Post-dorsal_thickness,lh_G_cingul-Post-ventral_thickness,lh_G_cuneus_thickness,lh_G_front_inf-Opercular_thickness,lh_G_front_inf-Orbital_thickness,lh_G_front_inf-Triangul_thickness,lh_G_front_middle_thickness,lh_G_front_sup_thickness,lh_G_Ins_lg&S_cent_ins_thickness,lh_G_insular_short_thickness,lh_G_occipital_middle_thickness,lh_G_occipital_sup_thickness,lh_G_oc-temp_lat-fusifor_thickness,lh_G_oc-temp_med-Lingual_thickness,lh_G_oc-temp_med-Parahip_thickness,lh_G_orbital_thickness,lh_G_pariet_inf-Angular_thickness".split(",")
norm_data = NormData.from_dataframe(
    name="full",
    dataframe=data,
    covariates=covariates,
    batch_effects=batch_effects,
    response_vars=response_vars,
)

# Leave two sites out for doing transfer and extend later
transfer_sites = ["Milwaukee_b", "Oulu"]
transfer_data, fit_data = norm_data.split_batch_effects(
    {"site": transfer_sites}, names=("transfer", "fit")
)

# Split into train and test sets
train, test = fit_data.train_test_split()
transfer_train, transfer_test = transfer_data.train_test_split()


In [13]:
# Create a NormConf object
save_dir = os.path.join(abs_path, "save_dir")
norm_conf = NormConf(
    savemodel=True,
    saveresults=True,
    save_dir=save_dir,
    inscaler="standardize",
    outscaler="standardize",
    basis_function="bspline",
    basis_function_kwargs={"order": 3, "nknots": 5},
)

Process: 754047 - Configuration of normative model is valid.


In [14]:
mu = make_prior(
    linear=True,
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 10.0)),
    intercept=make_prior(
        random=True,
        mu=make_prior(dist_name="Normal", dist_params=(0.0, 1.0)),
        sigma=make_prior(dist_name="HalfCauchy", dist_params=(0.5,)),
    ),
)
sigma = make_prior(
    linear=True,
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 10.0)),
    intercept=make_prior(dist_name="Normal", dist_params=(1.0, 1.0)),
    mapping="softplus",
    mapping_params=(0.0, 3.0),
)
# epsilon = make_prior(
#     dist_name="Normal",
#     dist_params=(0.0, 1.0),
# )
# delta = make_prior(
#     dist_name="Normal",
#     dist_params=(1.0, 2.0),
#     mapping="softplus",
#     mapping_params=(0.0, 3.0, 0.6),
# )

# Configure the HBRConf object
hbr_conf = HBRConf(
    draws=1500,
    tune=500,
    chains=4,
    pymc_cores=16,
    likelihood=NormalLikelihood(mu, sigma),
    nuts_sampler="nutpie",
)

In [16]:
new_hbr_model = NormHBR(norm_conf=norm_conf, reg_conf=hbr_conf)
sandbox_dir = os.path.join(resources_dir, "runner_dir")
os.makedirs(sandbox_dir, exist_ok=True)

In [17]:
runner = Runner(
    cross_validate=True,
    cv_folds=3,
    time_limit="00:10:00",
    job_type="slurm",
    log_dir=os.path.join(sandbox_dir, "log_dir"),
    temp_dir=os.path.join(sandbox_dir, "temp_dir"),
)

runner.fit_predict(new_hbr_model, train, test, observe=True)

Process: 754047 - 
Job Status Monitor:
--------------------------------------------
Job ID     Name     State     Time     Nodes
--------------------------------------------

Process: 754047 - 46816177   normative_job_0 RUNNING   0:30     dccn-c089
Process: 754047 - 46816178   normative_job_1 RUNNING   0:30     dccn-c089


KeyboardInterrupt: 